<a href="https://colab.research.google.com/github/kabilan942/Hugging-Face/blob/main/Hugging_Face_Summarization_of_CNN_Daily_Mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hugging Face - Summarization of CNN Daily Mail Dataset**

## **Installing transformers and Exploring Dataset**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 15.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.0 MB/s 
     |████████████████████████████████| 163 kB 71.7 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 19.7 MB/s 
     |████████████████████████████████| 115 kB 73.5 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 212 kB 73.6 MB/s 
     |████████████████████████████████| 127 kB 72.1 MB/s 
     |████████████████████████████████| 115 kB 76.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
# !git config --global user.email "kabilanp942@gmail.com"
# !git config --global user.name "Kabilan.P"

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
from datasets import load_dataset

datasets = load_dataset('cnn_dailymail','3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
datasets['train'][3]

{'article': 'WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush\'s colon on Saturday, and "none appeared worrisome," a White House spokesman said. The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said. Results are expected in two to three days. All were small, less than a centimeter [half an inch] in diameter, he said. Bush is in good humor, Stanzel said, and will resume his activities at Camp David. During the procedure Vice President Dick Cheney assumed presidential power. Bush reclaimed presidential power at 9:21 a.m. after about two hours. Doctors used "monitored anesthesia care," Stanzel said, so the president was asleep, but not as deeply unconscious as with a true general anesthetic. He spoke to first lady Laura Bush -- who is in Midland, Texas, celebrating her mother\'s birthday -- before and after the procedure, Stanzel said. Afterward, the president 

Since each article is accompanied by a short highlight, we can use the highlights as the target summaries for our model to learn from.

Therefore, the review information we are interested in is contained in the **'articles'** and 'highlights' columns.

In [6]:
def show_samples(dataset, num_samples=3, seed=42):
  sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))   
  # dataset['train'] used as Dataset for shuffle() and dataset["train"].shuffle(seed=seed) is used as Dataset for select()
  for example in sample:
    print()
    print(">> Article: {}".format(example['article']))
    print(">> Highlights: {}".format(example['highlights']))

show_samples(datasets)   # num_samples = 3 and seed = 42


>> Article: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detect

## **Preprocessing the data**

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Example of tokenization using 't5-small'

In [10]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [27, 1858, 1183, 8, 26049, 5880, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

['▁I', '▁loved', '▁reading', '▁the', '▁Hunger', '▁Games', '!', '</s>']

We apply truncation to both the article and their highlights to ensure we don’t pass excessively long inputs to our model.

The tokenizer has a **as_target_tokenizer()** function that allows you to tokenize the labels in parallel to the inputs.

This is typically done using a context manager inside a preprocessing function that first encodes the inputs, and then encodes the labels as a separate column.

In [12]:
# Values for max_input_length and max_target_length, which set the upper limits for how long our article and their highlights can be
# Since the article is typically much larger than the highlights, the values are scaled accordingly

max_input_length = 400
max_target_length = 100

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['article'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (highlights)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['highlights'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('highlights') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs



In [13]:
tokenized_dataset = datasets.map(preprocess_function, batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

We use batched=True in our Dataset.map() function above. This encodes the examples in batches of 1,000 (the default) and allows to make use of the multithreading capabilities of the fast tokenizers in Transformers.

## **Creating a Baseline**

A common baseline for text summarization is to simply take the first three sentences of an article, often called the lead-3 baseline. We could use full stops to track the sentence boundaries, but this will fail on acronyms like “U.S.”

Se we'll use the nltk library

In [14]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download the Punctuation rules:

In [15]:
import nltk 

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
from nltk.tokenize import sent_tokenize

# s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''
# sent_tokenize(s)
# ['Good muffins cost $3.88\nin New York.', 'Please buy me\ntwo of them.', 'Thanks.']

# join method
# myTuple = ("John", "Peter", "Vicky")
# x = "#".join(myTuple); print(x)
# OP: John#Peter#Vicky

def three_sentence_summary(text):   # function to extract 1st 3 sentences in the text
  return '\n'.join(sent_tokenize(text)[:3])   # first 3 sentence only

print(three_sentence_summary(datasets['train'][3]['article']))


WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and "none appeared worrisome," a White House spokesman said.
The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said.
Results are expected in two to three days.


Let's define a function which extracts these 'summaries' (1st 3 sentences) and compute rouge scores for them with 'summaries' as predictions and the 'highlights' as references in .compute() method

In [17]:
def evaluate_baseline(dataset, metric):
  summaries = [three_sentence_summary(text) for text in dataset['article']]
  return metric.compute(predictions=summaries, references=dataset['highlights'])

We can use this function to compute rouge scores over the validation set.

In [18]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=085684dc1988e82aa2ba0f0740cbcf8d862f94e05a4970ace60c9e865f234963
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [19]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 1.3 MB 34.0 MB/s 


In [20]:
import evaluate

rouge_score = evaluate.load('rouge')

In [21]:
import pandas as pd

# x = dict(name = "John", age = 36, country = "Norway")
# OP: {'name': 'John', 'age': 36, 'country': 'Norway'}

# round() -> returns a floating point number that is a rounded version of the specified number, with the specified number of decimals.
# round(5.76543, 2) -> 5.77

score = evaluate_baseline(datasets['validation'], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn]*100, 2)) for rn in rouge_names)

rouge_dict

{'rouge1': 39.5, 'rouge2': 17.56, 'rougeL': 24.95, 'rougeLsum': 36.12}

We can see that the rouge2 score is significantly lower than the rest; this likely reflects the fact that highlights are typically concise and so the lead-3 baseline is too verbose (using or expressed in more words than are needed).

## **Fine-tuning T5 with the Trainer API**

In [22]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)  # model_checkpoint = "t5-small"

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

We’ll need to generate summaries in order to compute ROUGE scores during training.

Hugging Face transformers provides Seq2SeqTrainingArguments and Seq2SeqTrainer classes that can do this automatically.

In [60]:
from transformers import Seq2SeqTrainingArguments

# parameters of Seq2SeqTrainingArguments
batch_size = 8
num_training_epochs = 1
logging_steps = len(tokenized_dataset["train"])//8

args = Seq2SeqTrainingArguments(evaluation_strategy='epoch',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-finetuned-cnn-dailymail-english")
                               # push_to_hub=True

# predict_with_generate argument has been set to True to indicate that we should generate summaries
# during evaluation so that we can compute ROUGE scores for each epoch
# The decoder performs inference by predicting tokens one by one, and this is implemented by the model’s 
# generate() method. Setting predict_with_generate=True tells the Seq2SeqTrainer to use that method for evaluation.
# save_total_limit = 3 -> because even the “small” version of T5 uses around a GB of hard drive space

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


For summarization evaluation is a bit more involved than simply calling rouge_score.compute() on the model’s predictions, since we need to decode the outputs and labels into text before we can compute the ROUGE scores.

In [24]:
import numpy as np

def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title')
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

Next, we need to define a data collator for our sequence-to-sequence task. Since T5 is an encoder-decoder Transformer model, one subtlety with preparing our batches is that during decoding we need to shift the labels to the right by one. This is required to ensure that the decoder only sees the previous ground truth labels and not the current or future ones, which would be easy for the model to memorize.

Hugging Face Transformers provides a **DataCollatorForSeq2Seq collator** that will dynamically pad the inputs and the labels for us. 

In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

First, we need to remove the columns with strings because the collator won’t know how to pad these elements

In [26]:
datasets['train'].column_names, len(datasets['train'].column_names)

(['article', 'highlights', 'id'], 3)

In [27]:
tokenized_dataset['train'].column_names, len(tokenized_dataset['train'].column_names)

(['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'], 6)

In [28]:
tokenized_dataset = tokenized_dataset.remove_columns(datasets['train'].column_names)

In [29]:
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

In [30]:
features = [tokenized_dataset['train'][i] for i in range(2)]
data_collator(features)

# Notice we have info ['input_ids', 'attention_mask', 'labels'] for 2 examples

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  301, 24796,  4170,     6,  2789,    41, 18844,    61,  1636,  8929,
         16023,  2213,  4173,  6324, 12591,    15, 11391,   592,    12,     3,
             9,  2196,  3996,  1755,   770,  8785,   591, 11039,   770,    61,
         13462,    38,     3,    88,  5050,   507,    30,  2089,     6,    68,
             3,    88, 10419,     7,     8,   540,   751,    31,    17,  4061,
             3,     9, 10783,    30,   376,     5,  4173,  6324, 12591,    15,
            38,  8929, 16023,    16,    96, 15537,   651, 16023,    11,     8,
          5197,    13,     8, 12308,   121,   304,     8, 19142,    13, 29517,
          6710,   343,     7,   300,     8,   296,     6,     8,  1021,  7556,
           845,     3,    88,    65,   150,  1390,    12,  9030,    17,   449,
           112,  1723,   550,    30,  1006,  2948,     6,  3281,    11, 17086,
          2251,     5,    96,   196,   278,    31,    17,   515,    12,    36,
            80,    13,   273,   151,  

The first example is longer than the second one, so the input_ids and attention_mask of the second example have been padded on the right with a [PAD] token (whose ID is 0). Similarly, we can see that the labels have been padded with -100s, to make sure the padding tokens are ignored by the loss function. And finally, we can see a new decoder_input_ids which has shifted the labels to the right by inserting a [PAD] token in the first entry.

In [31]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})

In [61]:
from transformers import Seq2SeqTrainer

# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["validation"], 
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric)

In [ ]:
trainer.train()

# notice with increasing epochs, the score becomes better (Try increasing no of epochs?)

***** Running training *****
  Num examples = 287113
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 35890


Epoch,Training Loss,Validation Loss


Saving model checkpoint to t5-finetuned-cnn-dailymail-english/checkpoint-500
Configuration saved in t5-finetuned-cnn-dailymail-english/checkpoint-500/config.json
Model weights saved in t5-finetuned-cnn-dailymail-english/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-finetuned-cnn-dailymail-english/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-finetuned-cnn-dailymail-english/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [t5-finetuned-cnn-dailymail-english/checkpoint-18000] due to args.save_total_limit
Saving model checkpoint to t5-finetuned-cnn-dailymail-english/checkpoint-1000
Configuration saved in t5-finetuned-cnn-dailymail-english/checkpoint-1000/config.json
Model weights saved in t5-finetuned-cnn-dailymail-english/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-finetuned-cnn-dailymail-english/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-finetuned-cnn-dailymail-english/checkpoi

Once the training is complete, we can see the final ROUGE scores by running trainer.evaluate(): 

In [ ]:
trainer.evaluate()

## **Using the Fine-Tuned Model**

Pre-Trained model -> "t5-small"

Pushing the model weights (after fine-tuning) to the hub

In [ ]:
trainer.push_to_hub(commit_message='Training_complete', tags='Summarization')

This will save the checkpoint and configuration files to output_dir, before uploading all the files to the Hub. By specifying the tags argument, we also ensure that the widget on the Hub will be one for a summarization pipeline instead of the default text generation one associated with the T5 architecture

The output from trainer.push_to_hub() is a URL to the Git commit hash

In [ ]:
from transformers import pipeline

hub_model = "t5-finetuned-cnn-dailymail-english"
summarizer = pipeline('summarization', model=hub_model)

print_summary() function to show the review, title, and generated summary together

In [ ]:
def print_summary(idx):
  review = datasets['test'][idx]['article']
  title = datasets['test'][idx]['highlights']
  summary = summarizer(datasets['test'][idx]['article'])[0]['summary_text']
  print(">>> Article: {}".format(review))
  print(">>> Highlights: {}".format(title))
  print(">>> Summary: {}".format(summary))

In [ ]:
print_summary(50)